In [1]:
import tensorflow as tf
import cv2 
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
import random
#%matplotlib inline

In [ ]:
##Load Image Create Dataset

In [ ]:
#%%bash
#sh ./*.sh

#INSERT HELPER SCRIPTS HERE

In [ ]:
imgDirNorm = './Normal/'
imgDirAnom = './Anomaly/'

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),0)
        if img is not None:
            images.append(img)
    return images

In [ ]:
def imrotate(img , angle , scale=1):
    (h, w) = img.shape
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, M, (h, w))

In [ ]:
#DATA AUGMENTATION  (OPTIONAL)
def augment_folder(folder):
    augmented_images = []
    images = load_images_from_folder(folder)

    for i in images:
        augmented_images.append(cv2.flip(i, 1))
        augmented_images.append(cv2.flip(i, 0))
        augmented_images.append(cv2.flip(i, -1))
        rotate90 = imrotate(i,90)
        rotate180 = imrotate(i,180)
        rotate270 = imrotate(i,270)
        augmented_images.append(cv2.flip(rotate90, 1))
        augmented_images.append(cv2.flip(rotate180, 0))
        augmented_images.append(cv2.flip(rotate270, -1))
        augmented_images.append(rotate90)
        augmented_images.append(rotate180)
        augmented_images.append(rotate270)
    
    return augmented_images    

In [ ]:
print('LOAD FROM DISK...........')

Anomaly = load_images_from_folder(imgDirAnom)
Normal = load_images_from_folder(imgDirNorm)

print('DISK LOAD COMPLETE!')

print('NORMAL IMAGES : ' , len(Normal))
print('ANOMALOUS IMAGES : ' , len(Anomaly))

print('IMAGE AUGMENT')
augmented_images = augment_folder(imgDirAnom)
Anomaly = Anomaly + augmented_images
print('IMAGE AUGMENT COMPLETE')

print('NORMAL IMAGES : ' , len(Normal))
print('ANOMALOUS IMAGES : ' , len(Anomaly))

print('DATA SHUFFLE')
Anomaly=random.shuffle(Anomaly)
Normal=random.shuffle(Normal)
print('SHUFFLE COMPLETE')

train_images=[]
train_labels=[]
test_images=[] 
test_labels=[]

print('LABLE CREATION')

ratio = len(Normal)//len(Anomaly)

i=0
j=0

while(i<len(Normal) and j<len(Anomaly)):
    
    if i%ratio:
        if (Normal[i].shape[0]==512):
            Normal[i] = cv2.resize(Normal[i]  , (1024 , 1024))  
        train_images.append(np.reshape(Normal[i],(1024,1024,1)))
        train_labels.append([0])

    if not i%ratio:
        if (Anomaly[j].shape[0]==512):
            Anomaly[j] = cv2.resize(Anomaly[j]  , (1024 , 1024))            
        train_images.append(np.reshape(Anomaly[j],(1024 , 1024 , 1)))
        train_labels.append([1])
        j+=1
        if (Normal[i].shape[0]==512):
            Normal[i] = cv2.resize(Normal[i]  , (1024 , 1024)) 
        train_images.append(np.reshape(Normal[i],(1024 , 1024 , 1)))
        train_labels.append([0])
        

    i+=1
print('LABELING COMPLETE')

class_names = ['Clean', 'Anomaly Present']

In [ ]:
train_images=np.array(train_images)/255.0
#train_labels=np.array(train_labels)/255.0
test_images=np.array(test_images)/255.0 
#test_labels=np.array(test_labels)/255.0

In [ ]:
#(train_images2, train_labels2), (test_images2, test_labels2) = datasets.cifar10.load_data()
#print ((train_images2[0]).shape)
#print (type(train_images2))
# Normalize pixel values to be between 0 and 1
#print(train_images2.shape)
#print(train_labels2.shape)

#train_images2, test_images2 = train_images2 / 255.0, test_images2 / 255.0


In [ ]:
test_images=train_images[2200:len(train_images)]
train_images=train_images[0:2200]

test_labels=train_labels[2200:len(train_labels)]
train_labels=train_labels[0:2200]

print((train_images[0]).shape)
print(type(train_images))
print(np.array(train_images).shape)
print(np.array(train_labels).shape)
print(len(test_images))
print(len(test_labels))

In [ ]:
'''class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()'''

In [3]:
inputshape = (1024, 1024, 1)
model = models.Sequential()
model.add(layers.Conv2D(64, (2, 2), activation='relu', input_shape=inputshape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (1, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (1, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (1, 1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 1023, 1023, 64)    320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 511, 511, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 511, 511, 64)      4160      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 255, 255, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 255, 255, 16)      1040      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 127, 127, 16)     

In [ ]:
'''img_file = './model_arch.png'
tf.keras.utils.plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)'''

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
'''plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)'''